# Introduction to Terraform

[Terraform](https://www.terraform.io/) is a powerful **Infrastructure as Code (IaC)** tool that allows cloud engineers to define, provision, and manage cloud infrastructure in a declarative configuration file. It is widely used to automate the creation and maintenance of cloud resources, such as virtual machines, databases, and storage accounts, across multiple cloud providers like Azure, AWS, and Google Cloud. Terraform works by using a configuration file (written in HashiCorp Configuration Language, HCL) that outlines the desired state of the infrastructure, which Terraform then compares against the current state to apply changes in an automated and consistent manner.  

## Why Terraform in a Cloud Engineer's Tool Stack?

Although you’ve already worked with Ansible, which is a great tool for configuration management and automating tasks across servers, Terraform focuses on provisioning infrastructure itself. Ansible excels at managing the state of an environment (such as installing software or configuring services), but when it comes to building, modifying, and managing cloud resources across providers, Terraform's **provider-agnostic** nature makes it a better fit. In fact, both tools are essential in a cloud engineer’s toolkit:

* Terraform: Used for infrastructure provisioning (setting up storage accounts, virtual networks, databases, etc.).
* Ansible: Used for configuration management and automation on top of that infrastructure (installing packages, managing services, etc.).

By using both tools together, cloud engineers can build and manage a full environment in a structured, automated, and repeatable way.

## Setting Up an Azure Storage Account with Terraform  

Before getting started, ensure you have the following installed:

* Terraform
* Azure CLI
* AWS CLI

You will also need the following:
* Azure Subscription
* AWS Access Key and Secret Key

This is a perfect task for Ansible, so go ahead and run this playbook to install the required tools:  

```yaml  
ansible-playbook environment_setup_playbook.yml  
source /opt/azure-cli-venv/bin/activate  
```  

### Step 1: Authenticating Terraform with Azure

First, authenticate your Terraform instance to interact with Azure by running:  

```bash
az login --tenant 5ca2bc70-353c-4d1f-b7d7-7f2b2259df68
```  

This command will open a browser window for you to log in to your Azure account. Once authenticated, you can proceed to the next step.  

### Step 2: Create an Azure Storage Account using Terraform  

Here’s a basic Terraform configuration file (main.tf) to create an Storage Account on an existing Resource Group:  

storage/azure/main.tf
```hcl
# Define the Azure Provider
provider "azurerm" {
  features {}
  subscription_id = "e0b9cada-61bc-4b5a-bd7a-52c606726b3b"
}

# Use the existing resource group that is not managed by Terraform:
data "azurerm_resource_group" "student_group" {
  name = "IE_ST_BCSAI_CLOUD_STUDENT"
}

# Storage Account
resource "azurerm_storage_account" "instructor_storage" {
  name                     = "instructoriac1"
  resource_group_name      = data.azurerm_resource_group.student_group.name
  location                 = data.azurerm_resource_group.student_group.location
  account_tier             = "Standard"
  account_replication_type = "LRS"
}
```

To create the Storage Account, change to the directory storage/azure and run the following commands:  

```bash
terraform init
```  

This command initializes the working directory containing the Terraform configuration files.  

```bash
terraform plan
```

This command creates an execution plan that shows what Terraform will do when you run `terraform apply`.  

```bash
terraform apply
```

This command applies the changes required to reach the desired state of the configuration.

```bash
terraform show
```  

This command shows the current state of the infrastructure.  

```bash
terraform destroy
```  

This command destroys the Terraform-managed infrastructure. Notice the Resource Group (IE_ST_BCSAI_CLOUD_STUDENT) is not destroyed, since it is being referenced via the data block, not created or managed by Terraform.

## Setting Up an AWS S3 Bucket with Terraform  

To create an S3 bucket on AWS, you need to authenticate Terraform with AWS and create a configuration file similar to the one used for Azure.  

### Step 1: Authenticating Terraform with AWS  

First, authenticate your Terraform instance to interact with AWS by running:  

```bash
aws configure
```  

This command will prompt you to enter your AWS Access Key ID, Secret Access Key, and default region.  

### Step 2: Create an AWS S3 Bucket using Terraform  

Here’s a basic Terraform configuration file (main.tf) to create an S3 bucket on AWS:  

storage/aws/main.tf
```hcl
# Define the AWS provider
provider "aws" {
  region = "us-east-1"  # Replace with your preferred region
}

# Create an S3 bucket
resource "aws_s3_bucket" "instructor_bucket" {
  bucket = "instructoriac1"

  tags = {
    Name        = "Instructor S3 Bucket"
    Environment = "Development"
  }
}

# Enable versioning for the S3 bucket
resource "aws_s3_bucket_versioning" "instructor_bucket_versioning" {
  bucket = aws_s3_bucket.instructor_bucket.id

  versioning_configuration {
    status = "Enabled"
  }
}
```  

To create the S3 bucket, change to the directory storage/aws and run the following commands:  

```bash
terraform init
```  

This command initializes the working directory containing the Terraform configuration files.  

```bash
terraform plan
```  

This command creates an execution plan that shows what Terraform will do when you run `terraform apply`.  

```bash
terraform apply
```  

This command applies the changes required to reach the desired state of the configuration.  

```bash
terraform show
```  

This command shows the current state of the infrastructure.  

```bash
terraform destroy
```  

This command destroys the Terraform-managed infrastructure.  


## Conclusion  

Terraform is a powerful tool for automating the provisioning and management of cloud infrastructure. By defining infrastructure as code, cloud engineers can create, modify, and manage cloud resources in a consistent and repeatable way. Terraform's provider-agnostic nature allows it to work across multiple cloud providers, making it a versatile tool for managing infrastructure in a multi-cloud environment. By combining Terraform with Ansible, cloud engineers can automate the full lifecycle of cloud resources, from provisioning infrastructure to configuring services and applications. Terraform is an essential tool in a cloud engineer's toolkit, enabling them to build and manage cloud environments efficiently and effectively.  

# Terraform vs. OpenTofu: The Battle for Infrastructure as Code  

The conflict between Terraform and [OpenTofu](https://opentofu.org/) began when HashiCorp, the company behind Terraform, switched the licensing model from an open-source **Mozilla Public License (MPL)** to the **Business Source License (BSL)** in mid-2023. This new license restricts how Terraform can be used, particularly in commercial contexts. Many in the open-source community saw this as a betrayal, given Terraform’s prominence as an open-source infrastructure-as-code (IaC) tool. The BSL is more restrictive, preventing companies from offering managed services based on Terraform without permission from HashiCorp.

In response, a group of developers and community members forked Terraform’s last open-source version, creating OpenTofu (initially called OpenTF). OpenTofu aims to maintain the open-source nature of the project and operates under the **Linux Foundation**. It promises feature parity with Terraform up to version 1.5.x, before HashiCorp’s licensing change. OpenTofu has attracted support from companies like Oracle and the Cloud Native Computing Foundation (CNCF), as the cloud-native ecosystem relies heavily on fully open-source tooling​ 

## Current Situation

As of early 2024, OpenTofu has reached general availability (GA) and is seen as a viable alternative to Terraform, especially for those who want to stay within the open-source ecosystem. HashiCorp, meanwhile, has faced pushback but remains the dominant player due to Terraform’s established user base and features added after the fork. However, the fork has gained significant attention from companies wary of the new BSL restrictions​.

There is also speculation about whether IBM, which [acquired HashiCorp in 2024](https://newsroom.ibm.com/2024-04-24-IBM-to-Acquire-HashiCorp-Inc-Creating-a-Comprehensive-End-to-End-Hybrid-Cloud-Platform), might reconsider returning Terraform to an open-source license. This is particularly relevant given IBM's emphasis on fully open-source tools​.

## Should Developers Be Worried?

For individual developers and non-commercial users, the situation doesn’t pose immediate concerns — Terraform remains free for personal use under the BSL. However, organizations that rely heavily on Terraform for commercial purposes may need to consider their options. If you’re using Terraform features introduced after version 1.5.x, OpenTofu may not be a drop-in replacement yet, but it’s expected to evolve quickly.

For companies, the bigger concern lies in HashiCorp’s future direction under IBM and how this could affect licensing further. Migrating to OpenTofu provides a safeguard against potential licensing changes down the line​.

## Articles for Further Reading

* [How HashiCorp’s License Shakeup Seeded an Open Source Rebel – The Register](https://www.theregister.com/2024/04/04/opentofu_on_forking_terraform/)  
* [Where do Terraform and OpenTofu go from here? – The Register](https://www.theregister.com/2024/05/24/opinion_column_terraform/)  
* [Comparing OpenTofu and Terraform – Ned In The Cloud](https://nedinthecloud.com/2024/01/22/comparing-opentofu-and-terraform/)  
* [Terraform fork OpenTofu is now ready for prime time – TechCrunch](https://techcrunch.com/2024/01/10/terraform-fork-opentofu-is-now-ready-for-prime-time/)  